In [ ]:
# Caso não tenha instalado, instalar o pmdarima e atualizar o statsmodels
#!pip install update statsmodels==0.12.1 
#%pip install pmdarima
import pandas as pd
import pmdarima as pm

# Carrega os dados
series = pd.read_csv('dataset.csv')

# Se necessário faz uma separação
df = series[:375]

# Calcula o melhor conjunto de parâmetros para o modelo ARIMA
pm.auto_arima(df['coluna'], start_p=0, start_q=0, max_q=6, seasonal=False, trace=True, maxiter=1000)

In [ ]:
# Previsão com o modelo Arima multi-step 
from pandas import read_csv
from statsmodels.tsa.arima.model import ARIMA
import numpy
from math import sqrt
from sklearn.metrics import mean_squared_error
from pandas.plotting import autocorrelation_plot
from pandas.plotting import lag_plot
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf

# Carrega os dados
series = read_csv('dataset.csv', header=0, index_col=0, squeeze=True, parse_dates=True)

# Visualiza o gráfico de autocorrelação, autocorrelação parcial e por último o padrão dos dados (linear ou não linear)
autocorrelation_plot(series)
plot_pacf(series)
lag_plot(series)

# Divide o conjunto e passa para X apenas os dados correspondentes ao conjunto de treinamento
X = series.values[:375]

# Faz o fit model
model = ARIMA(X, order=(6,0,3))
model_fit = model.fit()

# Faz a previsão de multiplos passos fora da amostra 
forecast = model_fit.forecast(steps=125, dynamic=True)

# Calcula o erro de previsão através da métrica RMSE entre o conjunto de teste e os dados previstos
rmse = sqrt(mean_squared_error(series[375:], forecast))
print('Test RMSE: %.3f' % rmse)

In [ ]:
# ESTE TRECHO DE CÓDIGO CALCULA O COEFICIENTE DE AUTOCORRELAÇÃO DAS OBSERVAÇÕES (O PADRÃO É PEARSON)
from pandas import DataFrame

teste = pd.read_csv('dataset.csv')

test = DataFrame(teste)

test.info()

shift_1 = test['coluna'].autocorr(lag=1)
print(shift_1)

In [ ]:
# Previsão de múltiplas etapas baseada na estratégia de múltiplas saídas -- LSTM empilhado
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
from pandas import Series
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.layers import Dropout
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

# Cria uma série diferenciada (torna estacionária)
def difference(dataset, interval=1):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return np.array(diff)

# Inverte a diferenciação dos dados
def inverse_difference(history, yhat, interval=1):
    return yhat + history[-interval]

# Carrega os dados
series = pd.read_csv('dataset.csv', header=0, index_col=0, squeeze=True, parse_dates=True)

# Pego apenas os valores da Series
raw_seq = series.values

# Passo a série original para a função que faz a diferenciação para depois entrar no modelo
#diffs = difference(raw_seq)


# Essa é uma das formas que eu transformo a série em array para depois fazer a diferenciação - Way 2
#x = np.array(series)
#diffs = np.diff(x, n=1)

# Escolhe o número de passos/etapas para a entrada e saída
n_steps_in, n_steps_out = 3, 125

# Separa em conjunto de teste e treino
#size = int(len(diffs) * 0.75)
#train, test = diffs[0:size], diffs[size:len(diffs)]
train = raw_seq[:375]

# Coloca a série treino para uma escala entre -1 e 1 para entrar no LSTM, para a função de ativação obter possível melhor desempenho
#scaler = MinMaxScaler(feature_range=(-1, 1))
#train = train.reshape(-1, 1)
#model_scaler = scaler.fit(train)
#scaled_data = model_scaler.transform(train)

# split into samples
X, y = split_sequence(train, n_steps_in, n_steps_out)

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

# Define o modelo a ser utilizado
model = Sequential()
model.add(LSTM(5, activation='tanh', return_sequences=True, input_shape=(n_steps_in, n_features)))
model.add(LSTM(5, dropout=0.2))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mean_squared_error')

# Faz o ajuste do modelo
model.fit(X, y, epochs=100, verbose=0, batch_size=15)

# demonstrate prediction
x_input = raw_seq[:3]
x_input = x_input.reshape((1, n_steps_in, n_features))

# Faço a previsão usando o modelo treinado
yhat = model.predict(x_input, verbose=0)

# Reverto a série para a escala original - Este está ligada a necessidade da rede
#yhat = scaler.inverse_transform(yhat)

# Reverto a diferenciação para poder calcular o erro - Este está ligado a estacionariedade da série
#yhat = inverse_difference(raw_seq, yhat, len(test))

# Essa uma das maneiras de reverter a série que foi transformada em estacionária - Way 2
#yhat = np.r_[raw_seq[375], yhat[0]].cumsum() 
#yhat = np.concatenate(([x[375]], yhat[0])).cumsum()

# Cria uma lista vazia para guardar os valores previstos
predictions = list()

# Usa o flatten para conseguir fazer o cálculo do erro de previsão
predictions = yhat.flatten().tolist()

# Calcula o erro de previsão comparando o conjunto de teste com os valores preditos
rmse = sqrt(mean_squared_error(series[375:], predictions))

# Exibe o valor do erro de previsão
print('Test RMSE: %.3f' % rmse)

# Plota o gráfico do conjunto de teste e as previsões
dias = range(0,125)
plt.plot(dias, series[375:], dias, predictions)
plt.legend(['Real','Previsto'])
plt.show()

# Salva uma imagem do resultado no formato eps
#plt.savefig('VazaoGasF12LSTM_multi764Steps', format='eps')

In [ ]:
# Previsão de múltiplas etapas baseada na estratégia recursiva -- LSTM empilhado
from math import sqrt
from numpy import array
from numpy import mean
from numpy import std
from pandas import DataFrame
from pandas import concat
from pandas import read_csv
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from matplotlib import pyplot

# Faz o split do dataset
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

# Transforma em aprendizado supervisionado
def series_to_supervised(data, n_in=1, n_out=1):
	df = DataFrame(data)
	cols = list()
	# Sequencia de entrada (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
	# Sequencia de previsão (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
	# Coloca tudo junto
	agg = concat(cols, axis=1)
	# Apaga linhas com valores Nan (caso o dataset tenha)
	agg.dropna(inplace=True)
	return agg.values

# Root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

# Diferencia o dataset
def difference(data, interval):
	return [data[i] - data[i - interval] for i in range(interval, len(data))]

# Faz o fit do modelo
def model_fit(train, config):
	# desempacota o config
	n_input, n_unit, n_epochs, n_batch, n_diff = config
	# Prepara os dados
	if n_diff > 0:
		train = difference(train, n_diff)
	data = series_to_supervised(train, n_in=n_input)
	train_x, train_y = data[:, :-1], data[:, -1]
	train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
	# Define o modelo
	model = Sequential()
	model.add(LSTM(n_unit, activation='tanh', return_sequences=True, input_shape=(n_input, 1)))
	model.add(LSTM(n_unit, dropout=0.2))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	# Faz o fit
	model.fit(train_x, train_y, epochs=n_epochs, batch_size=n_batch, verbose=0)
	return model

# Previsão do modelo
def model_predict(model, history, config):
	# desempacota o config
	n_input, _, _, _, n_diff = config
	# prepara os dados
	correction = 0.0
	if n_diff > 0:
		correction = history[-n_diff]
		history = difference(history, n_diff)
	x_input = array(history[-n_input:]).reshape((1, n_input, 1))
	# Faz a previsão 
	yhat = model.predict(x_input, verbose=0)
	return correction + yhat[0]

# Faz o loop no conjunto de teste e adiciona a previsão atual como entrada para a próxima
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# Faz o split dos dados
	train, test = train_test_split(data, n_test)
	# Ajusta o modelo
	model = model_fit(train, cfg)
	# O histórico itera sobre o conjunto de treino
	history = [x for x in train]
	# Caminha ao longo do conjunto de teste
	for i in range(len(test)):
		# Ajusta o modelo e faz as previsões sobre o histórico (treino+cada previsão)
		yhat = model_predict(model, history, cfg)
		# Armazena a previsão na lista de previsões
		predictions.append(yhat)
		# Adiciona a previsão atual no histórico para que sirva de entrada para a próxima previsão
		history.append(yhat)
	# Calcula o erro de previsão sobre os dados reais
	error = measure_rmse(test, predictions)
	print(' valor real x previsto > %.3f' % error)
	return error

# Repete a avaliação 
def repeat_evaluate(data, config, n_test, n_repeats=2):
	# Avalia o modelo n vezes
	scores = [walk_forward_validation(data, n_test, config) for _ in range(n_repeats)]
	return scores

# Resume o desempenho do modelo
def summarize_scores(name, scores):
	# Exibe o resumo do resultado
	scores_m, score_std = mean(scores), std(scores)
	print('%s: %.3f RMSE (+/- %.3f)' % (name, scores_m, score_std))
	# Exibe um bloxplot dos scores
	pyplot.boxplot(scores)
	pyplot.show()

series = read_csv('dataset.csv', header=0, index_col=0)
data = series.values
# Para o split dos dados
n_test = 7
# Define a configuração 
config = [3, 5, 140, 1, 0]
# grid search
scores = repeat_evaluate(data, config, n_test)
# Resume os resultados
summarize_scores('lstm', scores)